In [1]:
import numpy as np
import pandas as pd 
import torch
from torch import nn
from Config import CONFIG
Config = CONFIG()
time = np.loadtxt("../Data/AAL/100408_timeseries.txt")

In [2]:
torch.__version__

'2.1.1+cu121'

In [3]:
## Entry Serie 
time.shape

(1200, 116)

In [4]:
from Dataset import TimeSDataset
Dataset = TimeSDataset("../Data/AAL/","Gender",label_path="../Data/Behavioral-HCP.csv")

In [5]:
data , label = next(iter(Dataset))
data = torch.stack([data,data])
data.shape , label   ## (B , T_prim , V)

(torch.Size([2, 1200, 116]), tensor(1))

In [6]:
## Add CLFToken to our data . 
CLFToken = np.random.rand(1, Config.T_prim, 1)
print(f"First Shape: {CLFToken.shape}")

CLFToken = np.repeat(CLFToken, Config.B, axis=0)
print(f"After Repeat: {CLFToken.shape}")

data = np.concatenate([CLFToken, data], axis=-1)
print(f"Final Shape: {data.shape}")
data.shape

First Shape: (1, 1200, 1)
After Repeat: (2, 1200, 1)
Final Shape: (2, 1200, 117)


(2, 1200, 117)

In [7]:
label.shape

torch.Size([])

In [8]:
from SplitWindows import SplitWindows
import torch
SplitedSeries = SplitWindows(data)

In [9]:
SplitedSeries.shape   # ( B , T , V , P )

torch.Size([2, 105, 117, 20])

In [10]:
Config.T

105

In [11]:
from AttentionArea import TemporalAttention , SpatialAttention
TAttention = TemporalAttention (num_heads = 8 ,input_dim = Config.V * Config.K_E , hidden_dim=32 )

In [12]:
Temporal_output = TAttention(SplitedSeries)
# (B , T , V , K_E)
Temporal_output.shape

torch.Size([2, 105, 117, 40])

In [13]:
# Spatial_output = SAttention(Temporal_output)

In [14]:
Config.V * Config.K_E 


4680

In [15]:
from AttentionArea import DynamicMatrix
DM = DynamicMatrix()
weights = DM(Temporal_output)
weights.shape

torch.Size([2, 105, 117, 117])

In [16]:
from AttentionArea import SpatialAttention
ST = SpatialAttention()
SpatialOut = ST(Temporal_output)
SpatialOut.shape

torch.Size([2, 105, 117, 40])

In [17]:
from torch_geometric.nn import GCNConv 

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
weights[1][1][0][16]

tensor(0., grad_fn=<SelectBackward0>)

In [19]:
import torch
from torch_geometric.nn import GCNConv

# Assuming x is your node features with shape (B, T, V, P)
# and adj is your weighted adjacency matrix with shape (B, T, V, V)
Batch = []

gcn_layer = GCNConv(in_channels=Config.K_E, out_channels=Config.K_F)

# Iterate over each graph in the batch
for i in range(Config.B):
    # Extract the adjacency matrix for the current graph
    adj_i = weights[i]  # Shape (T, V, V)
    out = []
    for t in range(Config.T):
        # Get the edge indices and weights for the current time step
        edge_index = adj_i[t].nonzero(as_tuple=False).T  # Shape (2, E)
        edge_weights = adj_i[t][edge_index[0], edge_index[1]]  # Shape (E,)
        print(f"edge_index SHAPE : {edge_index.shape}")
        print(f"edge_weights SHAPE : {edge_weights.shape}")
        # Select the corresponding node features for the current time step
        x_t = SpatialOut[i][t]
        print(f"x_t SHAPE : {x_t.shape}")
        # Apply the GCN layer
        x_t = gcn_layer(x_t, edge_index, edge_weight=edge_weights)
        # Store the output for the current time step
        out.append(x_t)
    Batch.append(torch.stack(out))

# Stack the output to get the final representation
out = torch.stack(Batch)  # Shape will depend on how you structure the output

edge_index SHAPE : torch.Size([2, 4])
edge_weights SHAPE : torch.Size([4])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 1177])
edge_weights SHAPE : torch.Size([1177])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 3190])
edge_weights SHAPE : torch.Size([3190])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 205])
edge_weights SHAPE : torch.Size([205])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 146])
edge_weights SHAPE : torch.Size([146])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 12])
edge_weights SHAPE : torch.Size([12])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 49])
edge_weights SHAPE : torch.Size([49])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 92])
edge_weights SHAPE : torch.Size([92])
x_t SHAPE : torch.Size([117, 40])
edge_index SHAPE : torch.Size([2, 60])
edge_weights SHAPE : torch.Size([60])
x_t SHAPE : torch.Size([1

In [20]:
out.shape

torch.Size([2, 105, 117, 40])

In [21]:
### Write the Main Class
from SplitWindows import SplitWindows
from AttentionArea import TemporalAttention
from AttentionArea import SpatialAttention
from AttentionArea import DynamicMatrix
from torch_geometric.nn import GCNConv
from AttentionArea import AttentionBlock
from Config import CONFIG
from Dataset import TimeSDataset
import numpy as np
import pandas as pd 
import torch
from torch import nn
from torch_geometric.nn import GCNConv
Dataset = TimeSDataset("../Data/AAL/","Gender",label_path="../Data/Behavioral-HCP.csv")
Config = CONFIG()
data , label = next(iter(Dataset))
data = torch.stack([data,data])
class DynDeepNet(nn.Module):
    def __init__(self ):
        super().__init__()
        # Define Clf token
        self.CLFToken = np.random.rand(1, Config.T_prim, 1)
        # Define Temporal Attention
        self.TAttention = TemporalAttention (Config.NUM_H ,Config.V * Config.K_E , Config.HIDDEN_DIM)
        # Define Spatial Attention 
        self.SAttention = SpatialAttention()
        # Define Dynamic Matrix
        self.Dm = DynamicMatrix()
        # Define GCN Layer
        self.gcn_layer = GCNConv(in_channels=Config.K_E, out_channels=Config.K_F)
        # Define Final Encoder layers
        self.Transformer = nn.Sequential(*[AttentionBlock(Config.T * Config.K_F, Config.HIDDEN_DIM, Config.NUM_H, Config.DROP_OUT) for _ in range(Config.NUM_LAYER)])
        self.Mlp_head = nn.Sequential(
            nn.LayerNorm(Config.T * Config.K_F),
            nn.Linear(Config.T * Config.K_F, Config.NUM_CLASS)
            # torch.nn.Softmax(dim=1)
        )
        self.Dropout = nn.Dropout(Config.DROP_OUT)
        self.Flatten = nn.Flatten(2,-1)
    def forward(self,x):
        ### Input : (B , T_Prim , V)
        
        ## ADD CLF TOKEN
        self.CLFToken = np.repeat(self.CLFToken, Config.B, axis=0)
        x = np.concatenate([self.CLFToken, x], axis=-1)
        
        ## Convert Signal Serie to the Window Signal
        x = SplitWindows(x)  ## OutPut : (B , T , V ,P)
        ## Temporal Transformer
        x = self.TAttention(x)  ## OutPut : (B , T , V , K_E)
        ## Spatial Attention
        x = self.SAttention(x)    ## OutPut : (B , T , V , K_E)
        # Apply Drop out
        x = self.Dropout(x)
        ## Dynamic Matrix
        A = self.Dm(x)           ## OutPut  : (B , T , V , V)
        ## GCN Layer
        BatchList = []
        # Iterate over each graph in the batch
        for i in range(Config.B):
            # Extract the adjacency matrix for the current graph
            adj_i = A[i]  # Shape (T, V, V)
            out = []
            for t in range(Config.T):
                # Get the edge indices and weights for the current time step
                edge_index = adj_i[t].nonzero(as_tuple=False).T  # Shape (2, E)
                edge_weights = adj_i[t][edge_index[0], edge_index[1]]  # Shape (E,)
                # Select the corresponding node features for the current time step
                x_t = x[i][t]
                # Apply the GCN layer
                x_t = self.gcn_layer(x_t, edge_index, edge_weight=edge_weights)
                # Store the output for the current time step
                out.append(x_t)
            BatchList.append(torch.stack(out))
        
        # Stack the output to get the final representation
        x = torch.stack(BatchList)  ## OUTPUT : (B , T , V , K_F) 
        # Transpose
        x = x.transpose(1,2)  ## OUTPUT : (B , V , T , K_F)
        # Flatten
        x = self.Flatten(x)  ## OUTPUT  : (B , V , T * K_F)
        # Drop Out
        x = self.Dropout(x)
        # Apply Final Encoder
        x = x.transpose(0,1)
        x = self.Transformer(x)   ## OUTPUT : (V , B , T * K_F)
        # Apply Linear Head
        cls = x[0]
        return self.Mlp_head(cls)

Model = DynDeepNet()
        

In [22]:
out = Model(data)

In [23]:
out.shape

torch.Size([2, 2])

In [31]:
Model

DynDeepNet(
  (TAttention): TemporalAttention(
    (attention): AttentionBlock(
      (layer_norm_1): LayerNorm((4680,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): NonDynamicallyQuantizableLinear(in_features=4680, out_features=4680, bias=True)
      )
      (layer_norm_2): LayerNorm((4680,), eps=1e-05, elementwise_affine=True)
      (linear): Sequential(
        (0): Linear(in_features=4680, out_features=32, bias=True)
        (1): GELU(approximate='none')
        (2): Dropout(p=0.0, inplace=False)
        (3): Linear(in_features=32, out_features=4680, bias=True)
        (4): Dropout(p=0.0, inplace=False)
      )
    )
    (Conv1): Conv2d(20, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (Conv2): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (SAttention): SpatialAttention(
    (Softmax): Softmax(dim=1)
  )
  (Dm): DynamicMatrix(
    (ReLU): ReLU()
    (Softmax): Softmax(dim=1)
  )
  (gcn_layer): GCNCo

In [30]:
from torchinfo import summary
summary(Model,(1200,116))

RuntimeError: Failed to run torchinfo. See above stack traces for more details. Executed layers up to: []